In [2]:
import pennylane as qml
from pennylane import numpy as np
from mitiq import pec, Executor
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise
from qiskit.providers.fake_provider import *

n_wires = 3
n_layers = 1
template = qml.SimplifiedTwoDesign
weights_shape = template.shape(n_layers, n_wires)
w1, w2 = [2 * np.pi * np.random.random(s) for s in weights_shape]

# Describe noise
noise_strength = 0.01

def configured_backend():
    # backend = provider.get_backend("ibm_osaka") # uncomment this line to use a real IBM device
    backend = FakeManila()
    # backend.options.update_options(...)
    return backend

#Ideal device
dev_ideal = qml.device('default.mixed', wires=n_wires)
#noisy device
# dev_noisy = qml.transforms.insert(
#     dev_ideal,
#     qml.DepolarizingChannel,
#     noise_strength
# )
dev_noisy = qml.device("qiskit.remote", wires=n_wires, backend=configured_backend())

# The above circuit cannot be directly plugged into the mitiq thing, so we use this quantumTape version instead
ops = [
    template(w1, w2, wires=range(n_wires)),
    qml.adjoint(template(w1, w2, wires=range(n_wires))),
]
measurements = [qml.expval(qml.PauliZ(0))]
circuit = qml.tape.QuantumTape(ops)

# run mitiq PEC
def executor(tape, dev):
    new_tape = qml.tape.QuantumTape(
        tape.operations,
        tape.measurements + measurements,
        shots=tape.shots,
    )
    return qml.execute([new_tape], dev)[0]

executable = Executor(lambda tape: executor(tape, dev_noisy))

noise_level = 0.01
reps = represent_operations_in_circuit_with_local_depolarizing_noise(circuit, noise_level)
print(f"{len(reps)} OperationRepresentation objects produced, assuming {100 * noise_level}% depolarizing noise.")

# compare the circuits performance
ideal_value = executor(circuit, dev_ideal)
print(ideal_value)

noisy_value = executor(circuit, dev_noisy)
print(noisy_value)

pec_value = pec.execute_with_pec(circuit, executable, representations=reps)
print(pec_value)

16 OperationRepresentation objects produced, assuming 1.0% depolarizing noise.
1.0000000000000002
0.671875
1.0023101039831277


In [37]:
qml.about()

Name: PennyLane
Version: 0.33.1
Summary: PennyLane is a Python quantum machine learning library by Xanadu Inc.
Home-page: https://github.com/PennyLaneAI/pennylane
Author: 
Author-email: 
License: Apache License 2.0
Location: /Users/bigsad/Downloads/Algorithm-Research/Student-Hub/Indy-Ng/.venv/lib/python3.11/site-packages
Requires: appdirs, autograd, autoray, cachetools, networkx, numpy, pennylane-lightning, requests, rustworkx, scipy, semantic-version, toml, typing-extensions
Required-by: PennyLane-Lightning, PennyLane-qiskit

Platform info:           macOS-12.6-x86_64-i386-64bit
Python version:          3.11.6
Numpy version:           1.23.5
Scipy version:           1.10.1
Installed devices:
- default.gaussian (PennyLane-0.33.1)
- default.mixed (PennyLane-0.33.1)
- default.qubit (PennyLane-0.33.1)
- default.qubit.autograd (PennyLane-0.33.1)
- default.qubit.jax (PennyLane-0.33.1)
- default.qubit.legacy (PennyLane-0.33.1)
- default.qubit.tf (PennyLane-0.33.1)
- default.qubit.torch (Penn